# 3rd Excercise on Neural Networks & Intelligent Systems #
## Authors ##
### Leonidas Avdelas, 03113182 ###
### Konstantinos Skouras, 03113096 ###

## Team A61 ##

In [1]:
from deap import base, creator, tools, algorithms
import numpy as np
import math
import pandas as pd
from IPython.display import display
import time
import csv
#import multiprocessing

In [46]:
pd.set_option('display.max_rows', 300)

In [5]:
def print_logbook(path):
    df1 = pd.read_csv(path)
    display(df1)

## Part 1 ##


The function we want to optimize has 2 variables. We are also trying to find it's minima, which is the creterion we want to optimize.

In [6]:
numVariables = 2

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

Our initial values are in the range of $ -500\leq x \leq 500$.

In [7]:
toolbox = base.Toolbox()
#toolbox.register("map", pool.map)
toolbox.register( "InitialValue", np.random.uniform, -500, 500)
toolbox.register( "indiv", tools.initRepeat, creator.Individual, toolbox.InitialValue, numVariables)

In [8]:
toolbox.register( "population", tools.initRepeat, list , toolbox.indiv)

Our function is the negative Chen V Function: $ -f(X)= - (\frac{b}{b^2+\left(x_1-0.4x_2-0.1\right)^2}+\frac{b}{b^2+\left(2x_1+x_2-1.5\right)^2}) $, 

where:
$ b = 0.001,\\
-500\leq x_i\leq 500, i = 1,2 \\
f_{min}(X^*)= - 2000\\
x^*_i = (0.388888888888889,0.722222222222222)$

In [9]:
def evalChenVFunc(indiv):
    b = 0.001
    res = -(b/(b**2 + (indiv[0] - 0.4*indiv[1] - 0.1)**2) + b/(b**2 + (2*indiv[0] + indiv[1] - 1.5)**2))
    return (res,)

In [10]:
evalChenVFunc([0.38888888888888, 0.72222222222222])


(-2000.0000000000002,)

We create a penalty function to make sure that our atoms are inside the bounds.

In [11]:
MIN_BOUND = np.array(-500.0 * numVariables)
MAX_BOUND = np.array(500.0 * numVariables)

def feasible(indiv):
    if any(indiv < MIN_BOUND) or any(indiv > MAX_BOUND):
        return False
    return True

def distance(indiv) :
    dist = 0.0
    for i in range (len( indiv )) :
        penalty = 0
        if ( indiv [i] < MIN_BOUND) : penalty = -500.0 - indiv [i]
        if ( indiv [i] > MAX_BOUND) : penalty = indiv [i] - 500
        dist = dist + penalty
    return dist

In [12]:
toolbox.register( "evaluate", evalChenVFunc)
toolbox.decorate( "evaluate", tools.DeltaPenality (feasible, 7.0, distance))

## Possible Crossover Algorithms ##
* cxBlend
* cxSimulatedBinary
* cxSimulatedBinaryBounded

## Possible Mutation Algorithms ##
* mutGaussian
* mutShuffleIndexes

In [13]:
alpha = 0.3

def my_cx(ind1, ind2):
    (ind1, ind2) = tools.cxBlend(ind1, ind2, alpha)
    return ind1, ind2

In [14]:
mu = 0
sigma = 500
indpb = 0.05
tournsize = 3

In [15]:
toolbox.register( "mate", my_cx)
toolbox.register( "mutate", tools.mutGaussian, mu=mu, sigma=sigma, indpb=indpb)

In [16]:
toolbox.register("select", tools.selTournament, tournsize=tournsize)

In [17]:
def ea2_with_stats(strategy='eaSimple', cxpb=0.5, mutpb=0.2, ngens=50, mu=50, n=200, verbose=True):      
    pop = toolbox.population(n=n)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    if strategy == 'eaSimple':
        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngens, stats=stats, halloffame=hof, verbose=verbose)
    elif strategy == 'eaMuPlusLambda':
        pop, logbook = algorithms.eaMuPlusLambda(pop, toolbox, mu=mu, lambda_=n-mu, cxpb=0.5, mutpb=0.2, ngen=ngens, stats=stats, halloffame=hof, verbose=verbose)
    elif strategy == 'eaMuCommaLambda':
        pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=mu, lambda_=n-mu, cxpb=0.5, mutpb=0.2, ngen=ngens, stats=stats, halloffame=hof, verbose=verbose)
    else:
        return -1
    return pop, logbook, hof

In [18]:
def relative_criteria(delta, log, goal=-2000):
    successes = len(list(filter(lambda x: x <= (goal+delta), log.select("min"))))
    if(successes == 0):
        succ_gen = None
        succ_min = None
        succ_evals = None
    else:
        successes = 1
        min_list = log.select("min")
        for i in range(len(min_list)):
            if min_list[i] < goal+delta:
                succ_gen = i
                succ_min = min_list[i]
                succ_evals = log.select("nevals")[i]
                break
    return successes, succ_gen, succ_min, succ_evals

In [19]:
def change_operators(operator, op_type, val, rating):
    if operator=="mate":
        if op_type=="cxBlend":
            alpha = val
            toolbox.register( "mate", tools.cxBlend, alpha=alpha)
        elif op_type=="cxSimulatedBinary":
            eta = val
            toolbox.register("mate", tools.cxSimulatedBinary, eta=eta)
        else:
            return -1
    elif operator == "mutate":
        if op_type == "mutGaussian":
            mu, sigma, indpb = val
            toolbox.register( "mutate", tools.mutGaussian, mu=mu, sigma=sigma, indpb=indpb)
        elif op_type == "mutShuffleIndexes":
            indpb = val
            toolbox.register("mutate", tools.mutShuffleIndexes, indpb=indpb)
        else:
            return -1
    elif operator == "select":
        if op_type == "selTournament":
            tournsize = val
            toolbox.register("select", tools.selTournament, tournsize=tournsize)
        else:
            return -1
    else:
        return -1

In [36]:

def gather_results(max_rounds, gens, operators, path, mode, strategies=['eaSimple','eaMuPlusLambda','eaMuCommaLambda']):
    # max_rounds: int
    # max_gens: int
    operators_mate, operators_mutate, operators_select = operators
    
    with open(path, mode=mode) as f:
        f_writer = csv.writer(f, delimiter=',')
        if mode == 'w':
            f_writer.writerow(['operators', 'strategy', 'successes', 's.avg.min', 's.avg.evals', 's.avg.gens', 'avg.evals', 'avg.min', 'avg.time'])
        
        for operator_mate in operators_mate:
            print("Mating operator:")
            print(operator_mate)
            op_w = operator_mate[1] + operator_mate[3]
            change_operators(operator_mate[0], operator_mate[1], operator_mate[2], operator_mate[3])
            
            for operator_mutate in operators_mutate:
                print("Mutating operator:")
                print(operator_mutate)
                op_w += (' ,' + operator_mutate[1] + operator_mutate[3])
                change_operators(operator_mutate[0], operator_mutate[1], operator_mutate[2], operator_mutate[3])
                
                for operator_select in operators_select:
                    print("Selection operator:")
                    print(operator_select)
                    op_w += (' ,' + operator_select[1] + operator_select[3])
                    change_operators(operator_select[0], operator_select[1], operator_select[2], operator_select[3])
                    
                    for strategy in strategies:
                        print("Strategy: " + strategy)
                        running_min = 0.0
                        running_evals = 0.0
                        running_time = 0.0
                        successes = 0
                        running_s_min = 0.0
                        running_s_evals = 0.0
                        running_s_gens = 0.0

                        for round in range(max_rounds):

                            #print("Round %d of %d" % (round+1, max_rounds))
                            time1 = time.process_time()
                            pop, log, hof = ea2_with_stats(strategy=strategy, n=n, mu=mu, ngens=gens, cxpb=cxpb, mutpb=mutpb, verbose=False)
                            time2 = time.process_time()

                            running_min += hof[0].fitness.values[0]
                            running_evals += np.mean(log.select("nevals"))
                            running_time += (time2-time1)

                            s, s_gen, s_min, s_eval = relative_criteria(delta, log)

                            successes += s
                            running_s_min += (0 if (s_min == None) else s_min)
                            running_s_gens += (0 if (s_gen == None) else s_gen)
                            running_s_evals += (0 if (s_eval == None) else s_eval)

                        if strategy == 'eaSimple':
                            strat_w = strategy  + ' ' + str(n) + ' ' + str(cxpb) + ' ' + str(mutpb)
                        else:
                            strat_w = strategy  + ' ' + str(mu) + ' ' + str(l) + ' ' + str(cxpb) + ' ' + str(mutpb)

                        avg_min = running_min/max_rounds
                        avg_evals = running_evals/max_rounds
                        avg_time = running_time/max_rounds
                        s_avg_min = running_s_min/max_rounds
                        s_avg_gen = running_s_gens/max_rounds
                        s_avg_evals = running_s_evals/max_rounds

                        f_writer.writerow([op_w, strat_w,  str(successes), str(s_avg_min), str(s_avg_evals), str(s_avg_gen), str(avg_evals), str(avg_min), str(avg_time)])


In [21]:
operators_mate = [("mate", "cxBlend", 0.3, "Low"),  ("mate", "cxBlend", 0.7, "High"), ("mate", "cxSimulatedBinary", 1, "High"), ("mate", "cxSimulatedBinary", 10, "Low")]
operators_mutate = [("mutate", "mutGaussian", (0,500,0.05), "Low"), ("mutate", "mutGaussian", (0,500,0.4), "High"), ("mutate", "mutShuffleIndexes", 0.05, "Low"), ("mutate", "mutShuffleIndexes", 0.4, "High")]
operators_select = [("select", "selTournament", 3, "Low"), ("select", "selTournament", 10, "High")]

operators = (operators_mate, operators_mutate, operators_select)

In [23]:
delta = 1000
n = 200
mu = 50
l = n - mu
cxpb=0.5 
mutpb=0.2


gather_results(5, 50, operators, 'results_5_50.csv', 'w')

Mating operator:
('mate', 'cxBlend', 0.3, 'Low')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.05, 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.4, 'High')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')


In [24]:
print_logbook('results_5_50.csv')

,operators,strategy,successes,s.avg.min,s.avg.evals,s.avg.gens,avg.evals,avg.min,avg.time
0,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaSimple 200 0.5 0.2,2,-400.000000,17.0,17.0,120.984314,-407.309635,0.952005
1,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuPlusLambda 50 150 0.5 0.2,1,-200.000000,8.0,8.0,105.984314,-410.869915,0.793704
2,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuCommaLambda 50 150 0.5 0.2,2,-400.000000,16.8,16.8,107.082353,-781.509620,0.743189
3,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,0,0.000000,0.0,0.0,122.513725,-76.282824,1.069110
4,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,0,0.000000,0.0,0.0,106.039216,-255.218935,0.782760
5,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,1,-200.000000,10.0,10.0,106.678431,-601.772463,0.789417
6,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,2,-400.000000,15.6,15.6,122.270588,-793.959671,0.966645
7,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,2,-400.000000,15.2,15.2,106.796078,-600.133717,0.750969
8,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,2,-400.000000,17.0,17.0,106.996078,-1000.000000,0.749029
9,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,1,-200.000000,4.4,4.4,122.588235,-562.348864,1.080300


In [25]:
gather_results(5, 100, operators, 'results_5_100.csv', 'w')

Mating operator:
('mate', 'cxBlend', 0.3, 'Low')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.05, 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.4, 'High')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strateg

In [26]:
gather_results(5, 150, operators, 'results_10_150.csv', 'w')

Mating operator:
('mate', 'cxBlend', 0.3, 'Low')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.4), 'High')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.05, 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Str

In [27]:
gather_results(10, 100, operators, 'results_10_100.csv', 'w')

Mating operator:
('mate', 'cxBlend', 0.3, 'Low')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.4), 'High')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.05, 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Str

In [28]:
gather_results(10, 50, operators, 'results_10_50.csv', 'w')

Mating operator:
('mate', 'cxBlend', 0.3, 'Low')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.4), 'High')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.05, 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Str

In [29]:
gather_results(10, 150, operators, 'results_10_150.csv', 'w')

Mating operator:
('mate', 'cxBlend', 0.3, 'Low')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.4), 'High')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Mutating operator:
('mutate', 'mutShuffleIndexes', 0.05, 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Strategy: eaMuCommaLambda
Selection operator:
('select', 'selTournament', 10, 'High')
Strategy: eaSimple
Strategy: eaMuPlusLambda
Str

In [30]:
print_logbook('results_10_100.csv')

,operators,strategy,successes,s.avg.min,s.avg.evals,s.avg.gens,avg.evals,avg.min,avg.time
0,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaSimple 200 0.5 0.2,3,-300.000000,10.3,10.3,120.970297,-502.767433,1.742583
1,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuPlusLambda 50 150 0.5 0.2,2,-200.000000,10.5,10.5,106.132673,-366.749789,1.488029
2,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuCommaLambda 50 150 0.5 0.2,1,-100.000000,2.6,2.6,106.004950,-495.484461,1.488167
3,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,1,-100.000000,2.9,2.9,120.656436,-556.029049,2.103290
4,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,2,-200.000008,3.8,3.8,105.878218,-297.379179,1.575890
5,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,2,-200.000000,11.6,11.6,105.692079,-301.251300,1.653097
6,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,5,-500.000000,29.7,29.7,121.000000,-872.294913,1.763519
7,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,5,-500.000000,20.4,20.4,105.987129,-742.477864,1.484009
8,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,6,-600.000000,27.2,27.2,105.997030,-815.147247,1.484693
9,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,4,-400.000008,15.3,15.3,120.891089,-591.777902,2.172550


In [31]:
print_logbook('results_10_150.csv')

,operators,strategy,successes,s.avg.min,s.avg.evals,s.avg.gens,avg.evals,avg.min,avg.time
0,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaSimple 200 0.5 0.2,4,-400.000000,17.1,17.1,120.545033,-600.407859,2.621346
1,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuPlusLambda 50 150 0.5 0.2,1,-100.000175,3.7,3.7,105.500000,-422.122254,2.199201
2,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuCommaLambda 50 150 0.5 0.2,4,-400.000001,14.7,14.7,105.700000,-624.106008,2.219743
3,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,1,-100.000000,7.7,7.7,120.308609,-124.641208,3.135979
4,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,1,-100.000000,9.0,9.0,105.660927,-200.073072,2.335177
5,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,1,-100.000000,6.8,6.8,105.624503,-109.334586,2.339468
6,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,5,-500.000004,21.4,21.4,120.713245,-825.638145,2.635599
7,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,4,-400.000000,18.3,18.3,105.846358,-759.398811,2.224010
8,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,5,-500.000012,26.7,26.7,105.693377,-999.832854,2.202698
9,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,5,-500.000000,16.1,16.1,120.341722,-613.164897,3.168385


In [32]:
print_logbook('results_10_50.csv')

,operators,strategy,successes,s.avg.min,s.avg.evals,s.avg.gens,avg.evals,avg.min,avg.time
0,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaSimple 200 0.5 0.2,4,-400.000000,13.3,13.3,121.150980,-691.816228,0.950033
1,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuPlusLambda 50 150 0.5 0.2,0,0.000000,0.0,0.0,106.943137,-503.346280,0.746641
2,"cxBlendLow ,mutGaussianLow ,selTournamentLow",eaMuCommaLambda 50 150 0.5 0.2,2,-200.000000,7.7,7.7,107.052941,-638.416572,0.742961
3,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,0,0.000000,0.0,0.0,121.649020,-109.626874,1.059015
4,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,1,-100.000000,2.9,2.9,106.849020,-410.203737,0.789351
5,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,1,-100.000002,2.6,2.6,106.927451,-289.728458,0.826517
6,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,2,-200.000060,7.6,7.6,121.927451,-800.243649,0.904668
7,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuPlusLambda 50 150 0.5 0.2,0,0.000000,0.0,0.0,107.066667,-405.920939,0.747002
8,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaMuCommaLambda 50 150 0.5 0.2,3,-300.000001,11.5,11.5,106.647059,-900.018078,0.786208
9,"cxBlendLow ,mutGaussianLow ,selTournamentLow ,...",eaSimple 200 0.5 0.2,1,-100.000005,2.9,2.9,121.831373,-132.076436,1.063888


From the previous statestics, we come to the conclusion that the best results are from Simulated Binary High, mutation Gaussian Low, Simple Algorithm and small rounds on Tournaments, as the consistently get high results between number of populations and number of rounds.

In [35]:
operators_mate2 = [("mate", "cxSimulatedBinary", 1, "High")]
operators_mutate2 = [("mutate", "mutGaussian", (0,500,0.05), "Low")]
operators_select2 = [("select", "selTournament", 3, "Low")]

operators2 = (operators_mate2, operators_mutate2, operators_select2)

In [44]:
firstRun = True
for cxpb in np.arange(0.05, 0.9, 0.05):
    print(cxpb)
    for mutpb in np.arange(0.05, 0.9, 0.05):
        if firstRun is True:
            gather_results(10, 100, operators2, 'optimal_results.csv', 'w', ['eaSimple'])
            firstRun = False
        else:
            gather_results(10, 100, operators2, 'optimal_results.csv', 'a', ['eaSimple'])


0.05
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')

Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Stra

Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
0.3
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')


Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Stra

Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Stra

Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Stra

Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple
Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Stra

In [47]:
print_logbook('optimal_results.csv')

,operators,strategy,successes,s.avg.min,s.avg.evals,s.avg.gens,avg.evals,avg.min,avg.time
0,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.05,0,0.000000,0.0,0.0,21.479208,-0.902963,1.219613
1,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.1,0,0.000000,0.0,0.0,30.764356,-206.004448,1.271196
2,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.15000000000000002,0,0.000000,0.0,0.0,40.004950,-211.079253,1.320620
3,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.2,0,0.000000,0.0,0.0,49.537624,-321.766808,1.369685
4,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.25,2,-200.000000,12.1,17.3,58.651485,-313.943673,1.433232
5,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.3,0,0.000000,0.0,0.0,68.327723,-0.809899,1.489859
6,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.35000000000000003,0,0.000000,0.0,0.0,77.461386,-257.799949,1.540919
7,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.4,0,0.000000,0.0,0.0,87.396040,-200.662722,1.590477
8,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.45,1,-100.000000,10.2,8.1,97.036634,-359.921607,1.646585
9,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 200 0.05 0.5,0,0.000000,0.0,0.0,106.005941,-220.727664,1.690087


We got the best results for crossover prob = 0.85 and  mutation prob = 0.8

In [49]:
cxpb = 0.85
mutpb = 0.8
n = 2000
gather_results(1, 1000, operators2, 'final_results.csv', 'w', ['eaSimple'])

Mating operator:
('mate', 'cxSimulatedBinary', 1, 'High')
Mutating operator:
('mutate', 'mutGaussian', (0, 500, 0.05), 'Low')
Selection operator:
('select', 'selTournament', 3, 'Low')
Strategy: eaSimple


In [50]:
print_logbook('final_results.csv')

,operators,strategy,successes,s.avg.min,s.avg.evals,s.avg.gens,avg.evals,avg.min,avg.time
0,"cxSimulatedBinaryHigh ,mutGaussianLow ,selTour...",eaSimple 2000 0.85 0.8,1,-1420.123069,1942.0,42.0,1940.465534,-1420.123069,234.721133
